# Model for Count Data

Version 2.1 - 14-July-2023

by Luis A. Urso (compilation), USP-EALQ, INSPER, UNIMEP

Credits to: Prof. Luis Paulo Favero, USP-ESALQ


## Introduction

This scrip must be execute from top down, witout let any execution block behind, since there are futher steps those depend on the created variables and functions. 

The dataset used for this exercise is based in a real situation (see the reference)

Reference:
Fisman, R.; Miguel, E. Corruption, Norms, and Legal Enforcement: Evidence from Diplomatic Parking Tickets.  Journal of Political Economy, v. 15, n. 6, p. 1020-1048, 2007. https://www.journals.uchicago.edu/doi/abs/10.1086/527495


## 1. REQUIRED PACKAGES AND GLOBAL FUNCTIONS

### 1.1. PACKAGES

In [ ]:
## Required Packages Import

import pandas as pd
import numpy as np
from math import exp, factorial

import statsmodels.api as sm # Stats Models Package
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP

from scipy import stats
from scipy.stats import norm
from scipy.interpolate import interp1d

import seaborn as sns 
import matplotlib.pyplot as plt 

import warnings
warnings.filterwarnings('ignore')

### 1.2. GLOBAL FUNCTIONS

In [ ]:
## FUNCTION: LR TEST (LOG-LIK RATIO TEST) - as known as WILKS TEST
## Access the godness of fit for 2 competing model, specifically one found by 
## maximization against other with constraints. 
## It is expected a p-value < 0.05 to demostrate that the parameters are statistically significantt

def lrtest(modelos):
    modelo_1 = modelos[0]
    llk_1 = modelo_1.llnull
    llk_2 = modelo_1.llf
    
    if len(modelos)>1:
        llk_1 = modelo_1.llf
        llk_2 = modelos[1].llf
    LR_statistic = -2*(llk_1-llk_2)
    p_val = stats.chi2.sf(LR_statistic, 1)
    return round(LR_statistic,2), round(p_val,2)

## 2. INITIAL EXPLORATORY ANALYSIS

In [ ]:
## Load the Dataset CORRUPTION
## Makes basic verifications and Statistics

df_corruption = pd.read_csv('corruption.csv', delimiter=',')
df_corruption

## See Dataset Characteristics 
df_corruption.info()

## See Univariate Statistics 
df_corruption.describe()

In [ ]:
## Makes a Frequency Table for Dependent Dariable (Y) 'violations'
## Uses the function 'values_counts' from PANDAS, without normalization to generate the percentage counts

## Check Point: Observa that majority (52.3%) of the violations are 0 (Zero) which suggested Excess of Zeros and 
##              a ZIP or ZINB Model maybe more adequate. Later we will verify with detail. 

contagem = df_corruption['violations'].value_counts(dropna=False)
percent = df_corruption['violations'].value_counts(dropna=False, normalize=True)
pd.concat([contagem, percent], axis=1, keys=['contagem', '%'], sort=True)

In [ ]:
## See the Histogram of Dependent Variable (Y) 'violations'

## Check Point: See that a preliminary view of Y variable in the Histogram, suggests:
##              - Excess of Zeros (Zero-Inflated Models) - concentrate amount of zeros (as verified before)
##              - Overdispersion (Negative Binomial Models) - long tail on right od distribution 

plt.figure(figsize=(10,5))
sns.histplot(data=df_corruption, x="violations", bins=20, color='darkorchid')
plt.show()

In [ ]:
## Prelimar diagnostic to observe the equality (or proximity) between MEAN and VARIANCE of the 
## Dependente Variable (Y) 'violations'. 

## Check Point: As in this exercise it is observed a considerable difference (VARIANCE > MEAN),
##              the OVERDISPERSION is present. Therefore this is not the ultimate test, we will verify
##              with more preision ahead in the script


pd.DataFrame({'Mean':[df_corruption.violations.mean()],
              'Variance':[df_corruption.violations.var()]})


In [ ]:
## Check the behavior of variables 'corruption' (X) versus 'violations'(Y), BEFORE the law enforcement 
## (variable 'post'='no') and AFTER the law enforcement (variable 'post'='no') 

fig, axs = plt.subplots(ncols=2, figsize=(20,10), sharey=True)

fig.suptitle('Transit Violation in NY BEFORE and AFTER the Law Enforcement',
             fontsize = 20)

post = ['no','yes']

def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.02, point['y']+.1, str(point['val']))

for i, v in enumerate(post):
    df = df_corruption[df_corruption.post==v]
    df['violations'] = np.log(df.violations)
    df.loc[df['violations']==np.inf, 'violations'] = 0
    df.loc[df['violations']==-np.inf, 'violations'] = 0
    sns.regplot(data=df, x='corruption', y='violations',order=3, ax=axs[i],
                ci=False, color='darkorchid')
    axs[i].set_title(v)
    axs[i].set_ylabel("Transit Violations in NY (logs)", fontsize = 17)
    axs[i].set_xlabel("Countries Corruption Indexes", fontsize = 17)
    label_point(df.corruption, df.violations, df.code, axs[i])  

plt.show()

## 3. POISSON MODEL [POISSON]

### 3.1. POISSON DISTRIBUTION CONCEPTS

In [ ]:
## POISSON DISTRIBUTION CONCEPTUAL PART (ACADEMIC)

## References: 
## Fisman, R.; Miguel, E. Corruption, Norms, and Legal Enforcement:
## Evidence from Diplomatic Parking Tickets.
## Journal of Political Economy, v. 15, n. 6, p. 1020-1048, 2007.
## https://www.journals.uchicago.edu/doi/abs/10.1086/527495

## Creating a POISSON Distribution function based on determined values of LAMBDA and for M data point.

def poisson_lambda(lmbda,m):
    return (exp(-lmbda) * lmbda ** m) / factorial(m)

In [ ]:
## Preparing for plotting the different values of LAMBDA

m = np.arange(0,21)

lmbda_1 = []
lmbda_2 = []
lmbda_4 = []

for item in m:
    ## for lambda = 1
    lmbda_1.append(poisson_lambda(1,item))
    ## for lambda = 2
    lmbda_2.append(poisson_lambda(2,item))
    ## for lambda = 4
    lmbda_4.append(poisson_lambda(4,item))

## Temporary Data Frame with lambdas values 

df_lambda = pd.DataFrame({'m':m,
                          'lambda_1':lmbda_1,
                          'lambda_2':lmbda_2,
                          'lambda_4':lmbda_4})
df_lambda

In [ ]:
## Plot the graph with diferent values of lambda

from scipy.interpolate import interp1d

def smooth_line_plot(x,y):
    x_new = np.linspace(x.min(), x.max(),500)
    f = interp1d(x, y, kind='quadratic')
    y_smooth=f(x_new)
    return x_new, y_smooth

x_new, lambda_1 = smooth_line_plot(df_lambda.m, df_lambda.lambda_1)
x_new, lambda_2 = smooth_line_plot(df_lambda.m, df_lambda.lambda_2)
x_new, lambda_4 = smooth_line_plot(df_lambda.m, df_lambda.lambda_4)

plt.figure(figsize=(7,5))
plt.plot(x_new,lambda_1, linewidth=5, color='#440154FF')
plt.plot(x_new,lambda_2, linewidth=5, color='#22A884FF')
plt.plot(x_new,lambda_4, linewidth=5, color='#FDE725FF')
plt.xlabel('m', fontsize=10)
plt.ylabel('Probabilidades', fontsize=10)
plt.legend([r'$\lambda$ = 1',r'$\lambda$ = 2',r'$\lambda$ = 4'], fontsize=12)
plt.show


### 3.2. POISSON REGRESSION MODEL ESTIMATION

In [ ]:
## Makes a POISSON Regression using an open database called CORRUPTION. 

## The argument 'family=sm.families.Poisson()' of function 'smf.glm' defines the 
## estimation model. In our case is Poisson

modelo_poisson = smf.glm(formula='violations ~ staff + post + corruption',
                         data=df_corruption,
                         family=sm.families.Poisson()).fit()

## Verify the model parameters
modelo_poisson.summary()

In [ ]:
## TIP: Another way to se a more complete models is by using the function 'summary_col'

summary_col([modelo_poisson],
            model_names=["MODEL"],
            stars=True,
            info_dict = {
                'N':lambda x: "{0:d}".format(int(x.nobs)),
                'Log-lik':lambda x: "{:.2f}".format(x.llf)
        })


In [ ]:
## Check Point: Based on analysis All Prection Variables (X) are statistically different of 0 (zero)
##              considering a Confidence Index [CI] of 5% (ceteris paribus), therefore, can we assume 
##              that the POISSON model is the best to use ?
##              
##              Now we will check the OVERDISPERSION again to confirm and depending on the results, decide
##              if we will have to go for a Negative Binomial Model [NB2]

## OVERDISPESION CAMERON and TRIVEDI Test

## Reference: 
##CAMERON, A. C.; TRIVEDI, P. K. Regression-based tests for overdispersion in
##the Poisson model. Journal of Econometrics, v. 46, n. 3, p. 347-364, 1990.

## Steps:
## 1º : Estimate a POISSON Model (done in the steps before);
## 2º : Create a new (Y) variable (Y*) using the fitted values of POISSON Model used before.
## 3º : Estimate an auxiliary OLS Model, with the Y* dependent variable and without a INTERCEPT
##      formula='ystar ~ 0 + lambda_poisson
## 4º : Observe BETA parameter significance

## Adding the lambda POISSON (lambda_poisson) to the original dataframe:
df_corruption['lambda_poisson'] = modelo_poisson.fittedvalues
df_corruption

## Creating the Y* to the original dataframe. It will be called 'ystar':
df_corruption['ystar'] = (((df_corruption['violations']
                            -df_corruption['lambda_poisson'])**2)
                          -df_corruption['violations'])/df_corruption['lambda_poisson']
df_corruption

## Estimating the auxiliary OLS, without INTERCEPT - see that the intercept is 0:
modelo_auxiliar = smf.ols(formula='ystar ~ 0 + lambda_poisson',
                          data=df_corruption).fit()

## See the parameter os the Auxiliary OLS Model
modelo_auxiliar.summary()

## IMPORTANT
##
## EVALUATING THE RESULTS: If p-value of lambda_poisson parameter is lower than 0.05 (< 0.05)
##                         indicated that the OVERDISPERSION exists, then the POISSON model may not have 
##                         the maximum Log-Lik which suggestes that he estimation of a Negative Binomial [NB2] model
##                         will probably ofer a better maximization of the Log-Lik

In [ ]:
## For didatic purposes, lets makes some estimations with POISSON Model.
## What would be the expected amount of TRANSIT VIOLATIONS for a DIPLOMATIC STAFF with 23 people
## consindering BEFORE the LAW ENFORCEMENT for a country with CORRUPTION INDEX = 0.5 ?

modelo_poisson.predict(pd.DataFrame({'staff':[23],
                                     'post':['no'],
                                     'corruption':[0.5]}))


In [ ]:

## Now lets check AFTER the LAW ENFORCEMENT

modelo_poisson.predict(pd.DataFrame({'staff':[23],
                                     'post':['yes'],
                                     'corruption':[0.5]}))

## 4. NEGATIVE BINOMIAL MODEL [NB2]

### 4.1. NEGATIVE BINOMIAL DISTRIBUTION CONCEPTS

In [ ]:
## BINOMIAL NEGATIVE [NB2] DISTRIBUTION CONCEPTUAL PART (ACADEMIC)

## Creating a BINOMIAL NEGATIVE Distribution function based on determined values of THETA AND DELTA and for M data point.

#theta: FORM parameter for Poisson-Gama (binomial negative)
#delta: DECAY RATE parameter for Poisson-Gama (binomial negative)

def bneg(theta, delta, m):
    return ((delta ** theta) * (m ** (theta - 1)) * (exp(-m * delta))) / factorial(theta - 1)

In [ ]:
## Preparing for plotting the different values of THETA and DELTA

m = np.arange(1,21)

bneg_theta2_delta2 = []
bneg_theta3_delta1 = []
bneg_theta3_delta05 = []

for item in m:
    ## theta=2 and delta=2
    bneg_theta2_delta2.append(bneg(2,2,item))
    ## theta=3 and delta=1
    bneg_theta3_delta1.append(bneg(3,1,item))
    ## theta=3 and delta=0.5
    bneg_theta3_delta05.append(bneg(3,0.5,item))
   
# Creating a data frame variating from 1 to 20 and different values of theta and delta

df_bneg = pd.DataFrame({'m':m,
                        'bneg_theta2_delta2':bneg_theta2_delta2,
                        'bneg_theta3_delta1':bneg_theta3_delta1,
                        'bneg_theta3_delta05':bneg_theta3_delta05})

df_bneg

In [ ]:
## Plot the graph with diferent values of THETA and DELTA

def smooth_line_plot(x,y):
    x_new = np.linspace(x.min(), x.max(),500)
    f = interp1d(x, y, kind='quadratic')
    y_smooth=f(x_new)
    return x_new, y_smooth

x_new, bneg_theta2_delta2 = smooth_line_plot(df_bneg.m,
                                             df_bneg.bneg_theta2_delta2)
x_new, bneg_theta3_delta1 = smooth_line_plot(df_bneg.m,
                                             df_bneg.bneg_theta3_delta1)
x_new, bneg_theta3_delta05 = smooth_line_plot(df_bneg.m,
                                              df_bneg.bneg_theta3_delta05)

plt.figure(figsize=(7,5))
plt.plot(x_new,bneg_theta2_delta2, linewidth=5, color='#440154FF')
plt.plot(x_new,bneg_theta3_delta1, linewidth=5, color='#22A884FF')
plt.plot(x_new,bneg_theta3_delta05, linewidth=5, color='#FDE725FF')
plt.xlabel('m', fontsize=12)
plt.ylabel('Probabilities', fontsize=12)
plt.legend([r'$\theta$ = 2 and $\delta$ = 2',
            r'$\theta$ = 3 and $\delta$ = 1',
            r'$\theta$ = 3 and $\delta$ = 0.5'],
           fontsize=12)
plt.show

### 4.2. NEGATIVE BINOMIAL [NB2] REGRESSION MODEL ESTIMATION

In [ ]:
## Estimating a Binomial Negative [NB2] Model

## See the parameter 'family=sm.families.NegativeBinomial(alpha=2.0963)' of the function 
## 'smf.glm' that defines the estimation of a Binomial Negative Model Type NB2
## with a 'fi' ('alpha' no Python) equal to 2.0963. 'fi' is the inverse of FORM Parameter TETHA
## of Poisson-Gama Distribution (fi=1/theta)

modelo_bneg = smf.glm(formula='violations ~ staff + post + corruption',
                      data=df_corruption,
                      family=sm.families.NegativeBinomial(alpha=2.0963)).fit()

#Parâmetros do modelo
modelo_bneg.summary()

In [ ]:
## IMPORTANT : Defining the optimal aplha ('fi') that MAXIMIZES the Log-Lik

n_samples = 10000
alphas = np.linspace(0, 10, n_samples)
llf = np.full(n_samples, fill_value=np.nan)
for i, alpha in enumerate(alphas):
    try:
        model = smf.glm(formula = 'violations ~ staff + post + corruption',
                        data=df_corruption,
                        family=sm.families.NegativeBinomial(alpha=alpha)).fit()
    except:
        continue
    llf[i] = model.llf
alpha_otimo = alphas[np.nanargmax(llf)]
alpha_otimo

In [ ]:
## Plot the ALPHAs Simulation Results 

plt.plot(alphas, llf, label='Log-Likelihood')
plt.axvline(x=alpha_otimo, color='#440154FF',
            label=f'alpha: {alpha_otimo:0.5f}')
plt.legend()

In [ ]:
#### I M P O R T A N T ####

## IMPORTANT: Reestimates the Negative Binomial Model with the optimal alpha ('alpha_otimo')

modelo_bneg = smf.glm(formula='violations ~ staff + post + corruption',
                      data=df_corruption,
                      family=sm.families.NegativeBinomial(alpha=alpha_otimo)).fit()

## See the Model Parameters 
modelo_bneg.summary()


In [ ]:
## Compares the Models POISSON x NEGATIVE BINOMIAL (NB2)

## Check Point: See that the Log-Lik of the NB2 model is better (higher) than the POISSON MODEL

summary_col([modelo_poisson, modelo_bneg], 
            model_names=["Poisson","BNeg"],
            stars=True,
            info_dict = {
                'N':lambda x: "{0:d}".format(int(x.nobs)),
                'Log-lik':lambda x: "{:.2f}".format(x.llf),
                'Pseudo-R2':lambda x: "{:.4f}".format(x.pseudo_rsquared()),
        })

In [ ]:
## Log-Lik Ratio Test - expectec a p-value < 0.05 
## Comparing Significance of Poisson x BNEG

lrtest([modelo_poisson, modelo_bneg])

In [ ]:
## Graphic comparing the POISSON and NB2 Log-Liks
## See that NB2 has a better performance (higher Log-Lik)

## Define a Datafram with the Log-Liks for each model
df_llf = pd.DataFrame({'modelo':['Poisson','BNeg'],
                      'loglik':[modelo_poisson.llf, modelo_bneg.llf]})
df_llf

## Plot
fig, ax = plt.subplots(figsize=(7,5))

c = ['#440154FF', '#22A884FF']

ax1 = ax.barh(df_llf.modelo, df_llf.loglik, color = c)
ax.bar_label(ax1, label_type='center', color='white', fontsize=12)
ax.set_ylabel("Estimation", fontsize=10)
ax.set_xlabel("Log-Likehood", fontsize=10)

In [ ]:
## Comparing Prevision NB2 x POISSON

## What would be the expected amount of TRANSIT VIOLATIONS for a DIPLOMATIC STAFF with 23 people
## consindering BEFORE the LAW ENFORCEMENT for a country with CORRUPTION INDEX = 0.5 ?


## Model Poisson:
modelo_poisson.predict(pd.DataFrame({'staff':[23],
                                     'post':['no'],
                                     'corruption':[0.5]}))



In [ ]:
## Model NB2:
modelo_bneg.predict(pd.DataFrame({'staff':[23],
                                     'post':['no'],
                                     'corruption':[0.5]}))

In [ ]:

## Now lets check AFTER the LAW ENFORCEMENT

## Model Poisson:
modelo_poisson.predict(pd.DataFrame({'staff':[23],
                                     'post':['yes'],
                                     'corruption':[0.5]}))

In [ ]:
## Model NB2
modelo_bneg.predict(pd.DataFrame({'staff':[23],
                                     'post':['yes'],
                                     'corruption':[0.5]}))

In [ ]:
## Addint POISSON and NB2 Fitted Values to the Original Dataset 

df_corruption['fitted_poisson'] = modelo_poisson.fittedvalues
df_corruption['fitted_bneg'] = modelo_bneg.fittedvalues

df_corruption

In [ ]:
## Plot a comparission of POISSON x NB2 by STAFF (X) x VIOLATIONS (Y)

plt.figure(figsize=(3,3))
sns.relplot(data=df_corruption, x='staff', y='violations', color='black', height=8)
sns.regplot(data=df_corruption, x='staff', y='fitted_poisson', order=3,
            color='#440154FF')
sns.regplot(data=df_corruption, x='staff', y='fitted_bneg', order=3,
            color='#22A884FF')
plt.xlabel('Number of Diplomats (staff)', fontsize=12)
plt.ylabel('Unpaid Parking Violations (violations)', fontsize=12)
plt.legend(['Observed', 'Poisson', 'Fit Poisson', 'CI Poisson',
            'BNeg', 'Fit BNeg', 'CI BNeg'],
           fontsize=12)
plt.show

## 5. ZERO-INFLATED POISSON MODEL [ZIP]

### 5.1. ZERO-INFLATED POISSON DISTRIBUTION CONCEPTS

In [ ]:
## ZERO-INFLATED POISSON DISTRIBUTION CONCEPTS 

## Reference: 
## LAMBERT, D. Zero-inflated Poisson regression, with an application to defects
## in manufacturing. Technometrics, v. 34, n. 1, p. 1-14, 1992.

## Creates a function for ZIP , with lambda=1 and plogit=0,7
def zip_lambda1_plogit07(m):
    lmbda = 1
    plogit = 0.7
    
    if m == 0:
        return (plogit) + ((1 - plogit) * exp(-lmbda))
    else:
        return (1 - plogit) * ((exp(-lmbda) * lmbda ** m) / factorial(m))

In [ ]:
## Prepare the data for plotting

m = np.arange(0,21)

zip_lambda1_plogit07 = [zip_lambda1_plogit07(i) for i in m]

## Creates a dataframe with 'm' from 0 to 20

df_zip = pd.DataFrame({'m':m,
                       'zip_lambda1_plogit07':zip_lambda1_plogit07})
df_zip

In [ ]:
## Plotting ZIP Distribution 

def smooth_line_plot(x,y):
    x_new = np.linspace(x.min(), x.max(),500)
    f = interp1d(x, y, kind='quadratic')
    y_smooth=f(x_new)
    return x_new, y_smooth

x_new, zip_lambda1_plogit07 = smooth_line_plot(df_zip.m,
                                               df_zip.zip_lambda1_plogit07)

plt.figure(figsize=(7,5))
plt.plot(x_new,zip_lambda1_plogit07, linewidth=7, color="#440154FF")
plt.xlabel('m', fontsize=12)
plt.ylabel('Probabilities', fontsize=12)
plt.legend([r'ZIP: $\lambda$ = 1 and plogit = 0.7'],
           fontsize=12)
plt.show


### 5.2. ZERO-INFLATED POISSON MODEL ESTIMATION

In [ ]:
## Estimating ZIP Model

## Defining the dependent variable (Y) that is 'violations' (dataset 'df_corruption')
y = df_corruption.violations

## Defining the prediction variables (Xs)
x1 = df_corruption[['staff','post','corruption']]
X1 = sm.add_constant(x1)

## Defining the prediction variables that will be part of LOGIT (inflated) componente. 
x2 = df_corruption[['corruption']]
X2 = sm.add_constant(x2)

## For this model it is required to make the categorical variables a DUMMY. 
## Dummization for 'post' variable.
X1 = pd.get_dummies(X1, columns=['post'], drop_first=True)

## Impostant - Need to convert to float, else an it will throw an error in the fit function.
X1 = X1.astype(float) 

## Makes the ZIP Model using the ZeroInflatedPoisson from package 'Statsmodels'
## Argument: 'exog_infl' are the variables to consider in the LOGIT (Inflated)

modelo_zip = sm.ZeroInflatedPoisson(y, X1, exog_infl=X2,
                                     inflation='logit').fit()

## See the Model Parameters
modelo_zip.summary()

In [ ]:
## Plot a Graph to Compare Predicted Transit Violations x Real Transit Violations
## Using the fitted ZIP Model 

## Observations: See that naturaly the ZIP Model removes the outliers

zip_predictions = modelo_zip.predict(X1, exog_infl=X2)
predicted_counts = np.round(zip_predictions)
actual_counts = df_corruption['violations']

plt.figure(figsize=(10,5))
plt.plot(df_corruption.index, predicted_counts, 'go-',
         color='orange')
plt.plot(df_corruption.index, actual_counts, 'go-',
         color='#440154FF')
plt.xlabel('Observation', fontsize=12)
plt.ylabel('Transit Violations', fontsize=12)
plt.legend(['Predicted by ZIP', 'Real from Dataset'],
           fontsize=12)
plt.show()

In [ ]:
## Comparing the models performance POISSON x ZIP
## POISSON is always compared against ZIP

## Check Point: See that the Log-Lik for ZIP Model is better (higher than) the POISSON

summary_col([modelo_poisson, modelo_zip], 
            model_names=["Poisson","ZIP"],
            stars=True,
            info_dict = {
                'N':lambda x: "{0:d}".format(int(x.nobs)),
                'Log-lik':lambda x: "{:.2f}".format(x.llf),
                'Pseudo-R2':lambda x: "{:.4f}".format(x.pseudo_rsquared()),
        })

In [ ]:
## LR - likelihood ratio test to compared the models Log-Lik
## Comopare ZIP x POISSON Log-Liks. Expected p-value < 0.05 to confirm it is a Zero Inflated

lrtest([modelo_poisson, modelo_zip])

In [ ]:
## Plot a Graph to compare the Log-Liks Poisson, BNeg e ZIP
## See that the BNEG model still have better performant (higher Log-Lik) and the reason is
## associated to the fact that we have a distribution with EXCESS OF ZEROS, which
## sugested we may try a Zero-Inflated Negative Binomial Model (ZINB) that certainly 
## will offer a better Log-Lik.

##Defining a dataframe with the respective LL

df_llf = pd.DataFrame({'modelo':['Poisson','ZIP','BNeg'],
                      'loglik':[modelo_poisson.llf,
                                modelo_zip.llf,
                                modelo_bneg.llf]})

## Plotting 
fig, ax = plt.subplots(figsize=(7,5))

c = ["#440154FF", "#453781FF", "#22A884FF"]

ax1 = ax.barh(df_llf.modelo, df_llf.loglik, color = c)
ax.bar_label(ax1, label_type='center', color='white', fontsize=14)
ax.set_ylabel("Estimation", fontsize=12)
ax.set_xlabel("Log-Likehood", fontsize=12)

In [ ]:
## Comparing Prevision POISSON, NB2, ZIP

## What would be the expected amount of TRANSIT VIOLATIONS for a DIPLOMATIC STAFF with 23 people
## consindering BEFORE the LAW ENFORCEMENT for a country with CORRUPTION INDEX = 0.5 ?


## Model Poisson:
modelo_poisson.predict(pd.DataFrame({'staff':[23],
                                     'post':['no'],
                                     'corruption':[0.5]}))

In [ ]:
## Model NB2:
modelo_bneg.predict(pd.DataFrame({'staff':[23],
                                  'post':['no'],
                                  'corruption':[0.5]}))

In [ ]:
## Model ZIP
## IMPORTANT - Must keep the same parameter order used in the FIT function
modelo_zip.params

modelo_zip.predict(pd.DataFrame({'const':[1],
                                 'staff':[23],
                                 'corruption':[0.5],
                                 'post_yes':[0]}),
                   exog_infl=pd.DataFrame({'const':[1],
                                           'corruption':[0.5]}))

In [ ]:
## Now lets check AFTER the LAW ENFORCEMENT

## Model Poisson:
modelo_poisson.predict(pd.DataFrame({'staff':[23],
                                     'post':['yes'],
                                     'corruption':[0.5]}))

In [ ]:
## Model NB2:
modelo_bneg.predict(pd.DataFrame({'staff':[23],
                                  'post':['yes'],
                                  'corruption':[0.5]}))

In [ ]:
## Modelo ZIP:
modelo_zip.predict(pd.DataFrame({'const':[1],
                                 'staff':[23],
                                 'corruption':[0.5],
                                 'post_yes':[1]}),
                   exog_infl=pd.DataFrame({'const':[1],
                                           'corruption':[0.5]}))


## 6. ZERO-INFLATED NEGATIVE BINOMIAL [ZINB]

### 6.1. ZERO-INFLATED NEGATIVE BINOMIAL DISTRIBUTION CONCEPTS

In [ ]:
## ZINB DISTRIBUTION CONCEPT

## Function for ZINB Distributuion, withtheta = 2,
## delta = 2, plogit = 0.7 and lambda_bneg = 2

def zinb_theta2_delta2_plogit07_lambda2(m):
    lambda_bneg = 1
    plogit = 0.7
    theta = 2
    delta = 2
    if m == 0:
        return (plogit) + ((1 - plogit) *
                           (((1) / (1 + 1/theta * lambda_bneg)) ** theta))
    else:
        return (1 - plogit) * ((delta ** theta) * (m ** (theta - 1)) *
                               (exp(-m * delta))) / factorial(theta - 1)

In [ ]:
## Prepare the data to plot ZINB Distribution

m = np.arange(0,21)

zinb_theta2_delta2_plogit07_lambda2 = [zinb_theta2_delta2_plogit07_lambda2(i)
                                       for i in m]

#Criando um dataframe com m variando de 0 a 20

df_zinb = pd.DataFrame({'m':m,
                       'zinb_theta2_delta2_plogit07_lambda2':zinb_theta2_delta2_plogit07_lambda2})
df_zinb


In [ ]:
## Plot the ZINB Distribution

def smooth_line_plot(x,y):
    x_new = np.linspace(x.min(), x.max(),500)
    f = interp1d(x, y, kind='quadratic')
    y_smooth=f(x_new)
    return x_new, y_smooth

x_new, zinb_theta2_delta2_plogit07_lambda2 = smooth_line_plot(df_zinb.m,
                                                              df_zinb.zinb_theta2_delta2_plogit07_lambda2)

plt.figure(figsize=(7,5))
plt.plot(x_new,zinb_theta2_delta2_plogit07_lambda2, linewidth=7, color="red")
plt.xlabel('m', fontsize=10)
plt.ylabel('Probabilities', fontsize=10)
plt.legend([r'ZINB: $\lambda$$_{bneg}$ = 1, plogit = 0.7, $\theta$ = 2 and $\delta$ = 2'],
           fontsize=10)
plt.show

In [ ]:
#### NICE TO SEE ####

### PLOT A DISTRIBUTION COMPARISION BETWEEN: POISSON, BNEG, ZIP AND ZINB

def smooth_line_plot(x,y):
    x_new = np.linspace(x.min(), x.max(),500)
    f = interp1d(x, y, kind='quadratic')
    y_smooth=f(x_new)
    return x_new, y_smooth

x_new, zinb_theta2_delta2_plogit07_lambda2 = smooth_line_plot(df_zinb.m,
                                                              df_zinb.zinb_theta2_delta2_plogit07_lambda2)

plt.figure(figsize=(15,10))
plt.plot(x_new,lambda_1, linewidth=3, color='#404688FF')
plt.plot(x_new,lambda_2, linewidth=3, color='#2C728EFF')
plt.plot(x_new,lambda_4, linewidth=3, color='#20A486FF')
plt.plot(x_new,bneg_theta2_delta2, linewidth=3, color='#75D054FF')
plt.plot(x_new,bneg_theta3_delta1, linewidth=3, color='#C7E020FF')
plt.plot(x_new,bneg_theta3_delta05, linewidth=3, color='#FDE725FF')
plt.plot(x_new,zip_lambda1_plogit07, linewidth=5, color="#440154FF")
plt.plot(x_new,zinb_theta2_delta2_plogit07_lambda2, linewidth=7, color="red")
plt.xlabel('m', fontsize=10)
plt.ylabel('Probabilities', fontsize=10)
plt.legend([r'Poisson: $\lambda$ = 1',
            r'Poisson: $\lambda$ = 2',
            r'Poisson: $\lambda$ = 4',
            r'BNeg: $\theta$ = 2 and $\delta$ = 2',
            r'BNeg: $\theta$ = 3 and $\delta$ = 1',
            r'BNeg: $\theta$ = 3 and $\delta$ = 0.5',
            r'ZIP: $\lambda$ = 1 and plogit = 0.7',
            r'ZINB: $\lambda$$_{bneg}$ = 1, plogit = 0.7, $\theta$ = 2 and $\delta$ = 2'],
           fontsize=12)
plt.show

### 6.2. ZERO-INFLATED NEGATIVE BINOMIAL MODEL ESTIMATION

In [ ]:
## Estimating ZINB Model

## Defining the dependent variable (Y) that is 'violations' (dataset 'df_corruption')
y = df_corruption.violations

## Defining the prediction variables (Xs)
x1 = df_corruption[['staff','post','corruption']]
X1 = sm.add_constant(x1)

## Defining the prediction variables that will be part of LOGIT (inflated) componente. 
x2 = df_corruption[['corruption']]
X2 = sm.add_constant(x2)

## For this model it is required to make the categorical variables a DUMMY. 
## Dummization for 'post' variable.
X1 = pd.get_dummies(X1, columns=['post'], drop_first=True)

## Impostant - Need to convert to float, else an it will throw an error in the fit function.
X1 = X1.astype(float) 

## Makes the ZIP Model using the ZeroInflatedPoisson from package 'Statsmodels'
## Argument: 'exog_infl' are the variables to consider in the LOGIT (Inflated)

modelo_zinb = sm.ZeroInflatedNegativeBinomialP(y, X1, exog_infl=X2,
                                     inflation='logit').fit()

## See the Model Parameters
modelo_zinb.summary()

In [ ]:
## Plot a graph to compare the ZINB Estimated Transit Violations x the Real
## Transit Violation in the Original Dataset

## Checkpoint: See that the ZINB has a higher Log-Lik, so it will be the best model.

zinb_predictions = modelo_zinb.predict(X1, exog_infl=X2)
predicted_counts = np.round(zinb_predictions)
actual_counts = df_corruption['violations']

plt.figure(figsize=(9,7))
plt.plot(df_corruption.index, predicted_counts, 'go-',
         color='orange')
plt.plot(df_corruption.index, actual_counts, 'go-',
         color='#440154FF')
plt.xlabel('Observation', fontsize=10)
plt.ylabel('Real Transit Violations', fontsize=10)
plt.legend(['Estimated Transit Violation with ZINB', 'Real Transit Violations from Dataset'],
           fontsize=10)
plt.show()

In [ ]:
## Comparing Models BNeg e ZINB
## BNEG is always compared against ZINB

summary_col([modelo_bneg, modelo_zinb], 
            model_names=["BNeg","ZINB"],
            stars=True,
            info_dict = {
                'N':lambda x: "{0:d}".format(int(x.nobs)),
                'Log-lik':lambda x: "{:.2f}".format(x.llf),
                'Pseudo-R2':lambda x: "{:.4f}".format(x.pseudo_rsquared()),
        })

In [ ]:
## Verify the Lok-Lik Ratio (BNEG x ZINB)
## Expetected a p-value < 0.05 - Confirmation it is a Zero Inflated

lrtest([modelo_bneg, modelo_zinb])

In [ ]:
## Graph comparing All Models Lok-liks

## CHECK POINT : See that the ZINB Model has the higher Log-lik (close to 0), this indicates that this 
##               model offers the best performance and the BETAs + additional parameters will offer
##               good values for exploratory analysis and prediction. 

## Define a dataframe with all LLs
df_llf = pd.DataFrame({'modelo':['Poisson','ZIP','BNeg','ZINB'],
                      'loglik':[modelo_poisson.llf,
                                modelo_zip.llf,
                                modelo_bneg.llf,
                                modelo_zinb.llf]})
df_llf

## Plot the Lok-liks for each Model

fig, ax = plt.subplots(figsize=(7,5))

c = ["#440154FF", "#453781FF", "#22A884FF", "orange"]

ax1 = ax.barh(df_llf.modelo, df_llf.loglik, color = c)
ax.bar_label(ax1, label_type='center', color='white', fontsize=12)
ax.set_ylabel("Estimation", fontsize=10)
ax.set_xlabel("Log-Likehood", fontsize=10)

In [ ]:
## Comparing Previsions POISSON, NB2, ZIP, ZINB

## What would be the expected amount of TRANSIT VIOLATIONS for a DIPLOMATIC STAFF with 23 people
## consindering BEFORE the LAW ENFORCEMENT for a country with CORRUPTION INDEX = 0.5 ?


## Model Poisson:
modelo_poisson.predict(pd.DataFrame({'staff':[23],
                                     'post':['no'],
                                     'corruption':[0.5]}))

In [ ]:
## Model NB2
modelo_bneg.predict(pd.DataFrame({'staff':[23],
                                  'post':['no'],
                                  'corruption':[0.5]}))

In [ ]:
## Model ZIP
## Obs: Keep the same parameter order used in the ZIP Model FIT
modelo_zip.params

modelo_zip.predict(pd.DataFrame({'const':[1],
                                 'staff':[23],
                                 'corruption':[0.5],
                                 'post_yes':[0]}),
                   exog_infl=pd.DataFrame({'const':[1],
                                           'corruption':[0.5]}))

In [ ]:
## Modelo ZINB:
## Obs: Keep the same parameter order used in the ZINB Model FIT
modelo_zinb.params

modelo_zinb.predict(pd.DataFrame({'const':[1],
                                  'staff':[23],
                                  'corruption':[0.5],
                                  'post_yes':[0]}),
                    exog_infl=pd.DataFrame({'const':[1],
                                            'corruption':[0.5]}))

In [ ]:
## Now lets check AFTER the LAW ENFORCEMENT

## Model Poisson
modelo_poisson.predict(pd.DataFrame({'staff':[23],
                                     'post':['yes'],
                                     'corruption':[0.5]}))

In [ ]:
## Model NB2:
modelo_bneg.predict(pd.DataFrame({'staff':[23],
                                  'post':['yes'],
                                  'corruption':[0.5]}))

In [ ]:
## Model ZIP:
modelo_zip.predict(pd.DataFrame({'const':[1],
                                 'staff':[23],
                                 'corruption':[0.5],
                                 'post_yes':[1]}),
                   exog_infl=pd.DataFrame({'const':[1],
                                           'corruption':[0.5]}))

In [ ]:
## Model ZINB
modelo_zinb.predict(pd.DataFrame({'const':[1],
                                  'staff':[23],
                                  'corruption':[0.5],
                                  'post_yes':[1]}),
                    exog_infl=pd.DataFrame({'const':[1],
                                            'corruption':[0.5]}))

In [ ]:
## Add ZIP and ZINB Fitted Values to the Original Dataset

df_corruption['fitted_zip'] = modelo_zip.predict(X1, exog_infl=X2)
df_corruption['fitted_zinb'] = modelo_zinb.predict(X1, exog_infl=X2)
df_corruption

In [ ]:
## Plot a Graph with the fitted values for all models (POISSON, BNEG, ZIP e ZINB)
## considering 'violations' on Y axis, in function of 'staff'on X axis, and show the CI intervalc

plt.figure(figsize=(20,10))
sns.relplot(data=df_corruption, x='staff', y='violations', color='black', height=8)
sns.regplot(data=df_corruption, x='staff', y='fitted_poisson', order=3,
            color='#440154FF')
sns.regplot(data=df_corruption, x='staff', y='fitted_bneg', order=3,
            color='#22A884FF')
sns.regplot(data=df_corruption, x='staff', y='fitted_zip', order=3,
            color='#453781FF')
sns.regplot(data=df_corruption, x='staff', y='fitted_zinb', order=3,
            color='orange')
plt.xlabel('Number of Diplomats (staff)', fontsize=12)
plt.ylabel('Unpaid Parking Violations (violations)', fontsize=12)
plt.legend(['Observed', 'Poisson', 'Fit Poisson', 'CI Poisson',
            'BNeg', 'Fit BNeg', 'CI BNeg',
            'ZIP', 'Fit ZIP', 'CI ZIP',
            'ZINB', 'Fit ZINB', 'CI ZINB'],
           fontsize=10)
plt.show

In [ ]:
## Visualize the dataset with Fitted Values

df_corruption

In [ ]:
## Save the Dataset with Fitted Values to Excel

df_corruption.to_excel('corruption_fitted.xlsx', index=False)

In [ ]:
### END OF THE SCRIPT ###

## 7. Conclusion

This exercise was made for didatic purposes, so it shows in a evolutive way, how the Lok-Lik increases by
selection the proper distribution model. 

The final conclusion is that based on the distribution of the dependent variable (Y), the ZINB models offers a 
better fit and provided the Max Log-Lik. 